In [0]:
%run /Workspace/Users/themallpocaws@inteltion.com/Data-Integration-Scenarios/00_common/common_function

In [0]:
# Read JSON file from S3 (use option("multiline", "true") if your JSON is formatted across multiple lines)
json_path = "s3://tmg-poc-awsdb-apse1-stack-97db8-bucket/unity-catalog/catalog/poc_raw/forecast.json"
df_json = spark.read.option("multiline", "true").json(json_path)
display(df_json)

In [0]:
latitude = df_json.select("latitude").first()[0]
longitude = df_json.select("longitude").first()[0]
forecast_temp_unit = "Celsius"

# Zip the two arrays (time and temperature) into one array of structs
df_forecast = df_json.select(arrays_zip("hourly.time", "hourly.temperature_2m").alias("forecast"))
# Explode the array so each row corresponds to one forecast entry
df_forecast = df_forecast.select(explode("forecast").alias("fc"))

# Build the DataFrame with the desired schema
df = df_forecast.select(
lit(latitude).alias("LATITUDE"),
lit(longitude).alias("LONGITUDE"),
to_date(col("fc.time").substr(1, 10), "yyyy-MM-dd").alias("FORECAST_DATE"),
to_timestamp(col("fc.time"), "yyyy-MM-dd'T'HH:mm").alias("FORECAST_TIME"),
col("fc.temperature_2m").alias("FORECAST_TEMP")
).withColumn("FORECAST_TEMP_UNIT", lit(forecast_temp_unit))

# Cast columns as required
df = df.withColumn("LATITUDE", col("LATITUDE").cast(DecimalType(12, 2))) \
    .withColumn("LONGITUDE", col("LONGITUDE").cast(DecimalType(12, 2))) \
    .withColumn("FORECAST_TEMP", col("FORECAST_TEMP").cast(DecimalType(12, 2)))
# Write to Databricks SQL table
# df.write.mode("overwrite") \
# .saveAsTable("POC_STG.POC.FORECAST")